In [4]:
import pandas
import requests
import datetime
import time
import json
from dateutil import parser

In [5]:
#Different type of Ripple Transactions...
#https://ripple.com/build/transactions/
#denomination of XRP is 1,000,000 units to 1 XRP

In [6]:
#https://ripple.com/build/data-api-v2/#get-transactions
url_endpoint = "https://data.ripple.com/v2/transactions/"
starting_datetime = datetime.datetime(2018,1,10,10,0)
iso_starting_datetime = starting_datetime.isoformat()
ending_timestamp = datetime.datetime(2018,1,10,10,10)
iso_ending_timestamp = ending_timestamp.isoformat()
total_api_calls = 100

In [7]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

In [8]:
params = {"start": iso_starting_datetime , "end" : iso_ending_timestamp, "limit" : 100}
resp = requests.get(url_endpoint, params=params)
resp_dict = json.loads(resp.content)

In [9]:
resp_dict

{u'count': 100,
 u'marker': u'20180110100002|000035711851|00025',
 u'result': u'success',
 u'transactions': [{u'date': u'2018-01-10T10:00:00+00:00',
   u'hash': u'8888BF6F8670CEC23F91A878ADC7083085FB140C57DB56B514BD8E68885E3A0D',
   u'ledger_hash': u'E44A0E0295B18B5C86C71F1BCEB2B956D02790C7CA0A0A55CCE0F92F48C47B62',
   u'ledger_index': 35711849,
   u'meta': {u'AffectedNodes': [{u'ModifiedNode': {u'FinalFields': {u'Account': u'r9R23ueipdfZ69SADjAMHFa38ewiXzxZ2P',
        u'Balance': u'40000779644',
        u'Flags': 0,
        u'OwnerCount': 3,
        u'Sequence': 29374},
       u'LedgerEntryType': u'AccountRoot',
       u'LedgerIndex': u'6E3ED26E89AF949E62B797E92C32221035E7AB87A7D9EC5615F17BFA78A7FD23',
       u'PreviousFields': {u'Balance': u'40000779656', u'Sequence': 29373},
       u'PreviousTxnID': u'6C91BFC80E267B6D656280558BC659FCA05FCAB1BDE3609B541A13EACCCF72E7',
       u'PreviousTxnLgrSeq': 35711815}}],
    u'TransactionIndex': 0,
    u'TransactionResult': u'tesSUCCESS'},
   u

In [10]:
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

def process_transaction_batch(transactions):
    '''
    Goes through a list of transactions from the Ripple transactions api.
    Returns a list of processed transactions.
    '''
    processed_txs = []

    for n, i in enumerate(transactions):
        tx_datetime = parser.parse(i['date'])
        tx_formatted_date = tx_datetime.strftime("%Y-%m-%d %H:%M:%S")
        unix_timestamp = int(time.mktime(tx_datetime.timetuple()))

        next_tx = {'datetime': tx_formatted_date, 'unixtimestamp': unix_timestamp, 'network': "Ripple"}

        #Unique Hash
        next_tx['txhash'] = i['hash']

        #Transaction Type
        transaction_type = i['tx']['TransactionType']
        next_tx['txtype'] = transaction_type

        #Fee
        next_tx['txfee'] = '{0:f}'.format(float(i['tx']['Fee'])/float(1000000))
        next_tx['txfeedenom'] = 'XRP'

        if transaction_type == "OfferCreate":      

            #Sender Pays in not XRP, and receives not XRP
            if type(i['tx']["TakerGets"]) == type({}) and type(i['tx']["TakerPays"]) == type({}): 

                #Account Sending
                next_tx['sender'] = i['tx']['Account']

                #Amount Sent
                next_tx['valuesent'] = i['tx']['TakerGets']['value']

                #Amount Sent Denomination
                next_tx['valuesentdenom'] = i['tx']['TakerGets']['currency']

                #Account Receiving
                next_tx['receiver'] = "Unknown"

                #Amount Received
                next_tx['valuereceived'] = i['tx']['TakerPays']['value']

                #Received Denomination
                next_tx['valuereceiveddenom'] = i['tx']['TakerPays']['currency']


            #Sender Pays in XRP, but Receives not XRP    
            elif type(i['tx']["TakerGets"]) != type({}) and type(i['tx']["TakerPays"]) == type({}): 
                #Account Sending
                next_tx['sender'] = i['tx']['Account']

                #Amount Sent
                next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['TakerGets'])/float(1000000))

                #Amount Sent Denomination
                next_tx['valuesentdenom'] = 'XRP'

                #Account Receiving
                next_tx['receiver'] = "Unknown"

                #Amount Received
                next_tx['valuereceived'] = i['tx']['TakerPays']['value']

                #Received Denomination
                next_tx['valuereceiveddenom'] = i['tx']['TakerPays']['currency']

            #Sender Pays in not XRP and receives XRP
            elif type(i['tx']["TakerGets"]) == type({}) and type(i['tx']["TakerPays"]) != type({}): 
                #Account Sending
                next_tx['sender'] = i['tx']['Account']

                #Amount Sent
                next_tx['valuesent'] = i['tx']['TakerGets']['value']

                #Amount Sent Denomination
                next_tx['valuesentdenom'] = i['tx']['TakerGets']['currency']

                #Account Receiving
                next_tx['receiver'] = "Unknown"

                #Amount Received
                next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['TakerPays'])/float(1000000))

                #Received Denomination
                next_tx['valuereceiveddenom'] = 'XRP'

            #Sender Pays in XRP and Receives XRP
            else:

                #Account Sending
                next_tx['sender'] = i['tx']['Account']

                #Amount Sent
                next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['TakerGets'])/float(1000000))

                #Amount Sent Denomination
                next_tx['valuesentdenom'] = 'XRP'

                #Account Receiving
                next_tx['receiver'] = "Unknown"

                #Amount Received
                next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['TakerPays'])/float(1000000))

                #Received Denomination
                next_tx['valuereceiveddenom'] = 'XRP'

        elif transaction_type == 'Payment':
                #Account Sender
                next_tx['sender'] = i['tx']['Account']

                #Account Receiving
                next_tx['receiver'] = i['tx']['Destination']
                
                #Sometimes payments are in amounts other than XRP
                if type(i['tx']['Amount']) == type({}):
                    #Amount Received
                    next_tx['valuereceived'] = i['tx']['Amount']['value']
                    
                    #Received Denomination
                    next_tx['valuereceiveddenom'] = i['tx']['Amount']['currency']

                    #Amount Sent
                    next_tx['valuesent'] = i['tx']['Amount']['value']
                    
                    #Amount Sent Denomination
                    next_tx['valuesentdenom'] = i['tx']['Amount']['currency']
                
                else:
                    #Amount Received
                    next_tx['valuereceived'] = '{0:f}'.format(int(i['tx']['Amount'])/float(1000000))
                    
                    #Received Denomination
                    next_tx['valuereceiveddenom'] = 'XRP'

                    #Amount Sent
                    next_tx['valuesent'] = '{0:f}'.format(int(i['tx']['Amount'])/float(1000000))
                    
                    #Amount Sent Denomination
                    next_tx['valuesentdenom'] = 'XRP'
                
        processed_txs.append(next_tx)

    return processed_txs

In [11]:
processed_txs = []
params = {"start": iso_starting_datetime , "end" : iso_ending_timestamp, "limit" : 100}

for i in range(total_api_calls):
    if i%10 == 0:
        print "Number of Cycles: " + str(i)
        print datetime.datetime.now()
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content)
    transactions = resp_dict['transactions']
    processed_txs = processed_txs + process_transaction_batch(transactions)
    if 'marker' in resp_dict:
        params['marker'] = resp_dict['marker']
    else:
        break

Number of Cycles: 0
2018-02-09 15:23:33.871000
Number of Cycles: 10
2018-02-09 15:23:51.162000
Number of Cycles: 20
2018-02-09 15:24:07.956000
Number of Cycles: 30
2018-02-09 15:24:23.521000
Number of Cycles: 40
2018-02-09 15:24:41.259000
Number of Cycles: 50
2018-02-09 15:24:58.104000
Number of Cycles: 60
2018-02-09 15:25:16.139000
Number of Cycles: 70
2018-02-09 15:25:34.336000
Number of Cycles: 80
2018-02-09 15:25:55.033000
Number of Cycles: 90
2018-02-09 15:26:14.632000


In [12]:
transactions[0]

{u'date': u'2018-01-10T10:10:00+00:00',
 u'hash': u'7B354D35A9A3BFB1C3702B38BE17220FECD5296AB14C873B0641F8A20C64F575',
 u'ledger_hash': u'69F909D0770FD9966B339CB6D0EC4CF6D0225FFA8C55E5815CB231E5CB553436',
 u'ledger_index': 35712026,
 u'meta': {u'AffectedNodes': [{u'DeletedNode': {u'FinalFields': {u'ExchangeRate': u'4C1C5041D703A73C',
      u'Flags': 0,
      u'RootIndex': u'14B232AD91700713C3570549D6E9FBD77878255EE4AE06034C1C5041D703A73C',
      u'TakerGetsCurrency': u'0000000000000000000000000000000000000000',
      u'TakerGetsIssuer': u'0000000000000000000000000000000000000000',
      u'TakerPaysCurrency': u'0000000000000000000000004C54430000000000',
      u'TakerPaysIssuer': u'6F2531F2B8CDB96D6D986D9D75CC0156DF2C5387'},
     u'LedgerEntryType': u'DirectoryNode',
     u'LedgerIndex': u'14B232AD91700713C3570549D6E9FBD77878255EE4AE06034C1C5041D703A73C'}},
   {u'CreatedNode': {u'LedgerEntryType': u'DirectoryNode',
     u'LedgerIndex': u'14B232AD91700713C3570549D6E9FBD77878255EE4AE06034C

In [13]:
df = pandas.DataFrame.from_dict(processed_txs)

df.to_csv("Ripple_20180110_10am_transactions.csv")

In [34]:
len(processed_txs)

10000